In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from more_itertools import set_partitions

In this notebook, we calculate which coalition structures (collections of federating coalitions) are stable, for various definitions of stability. First, we need to calculate the error each player experiences: for this, we use the calculate_means() function we built previously. 

In [2]:
means_dist = stats.norm(loc = 0, scale = 1)
variance_dist = stats.beta(a=8, b=2, scale = 50/4)
print(means_dist.var())
print(variance_dist.mean())

1.0
10.0


In [3]:
def calculate_means(var = means_dist.var(), mue = variance_dist.mean(), 
                    n_list = [10, 20, 30], w_best = False, w_list = [0.2, 0.4, 0.6], v_best = False,
                    v_mat = [[0.1, 0.6, 0.3], [0.2, 0.8, 0.0], [0.3, 0.5, 0.2]]):
    '''
    Calculate exact error for mean estimation.  

    Args:
        var: variance of true mean distributions
        mue: mean of true error distribution. 
        n_list: a list of length M (number of players) with the number of samples each has.
        w_best: boolean, if true, calculates error given optimal values for w
        w_list: if w_best is false, a list of w-weights (in [0, 1]) for coarse-grained federation.
        v_best: boolean, if true, calculates error given optimal values for v
        v_mat: a matrix (list of lists) of weights each player uses in fine-grained federation: the rows sum up 
               to 1.
    Returns:
        dataframe with average error for each player, for: local, uniform, coarse-grained, and fine-grained 
        federation.  
    '''
    # dataframe for storing error
    player_error = pd.DataFrame(data = 0.0, index = ['local', 'uniform', 'coarse', 'fine'], 
                                columns = range(len(n_list)))
    N = sum(n_list)
    
    # for each player, calculate their true error 
    for j, n in enumerate(n_list):
        
        # local
        player_error.loc['local'][j] = mue/n
        
        sumsquares = sum([nval**2 for nval in n_list]) - n**2 + (N-n)**2
        
        # uniform
        player_error.loc['uniform'][j] = mue/N + sumsquares * var/(N**2)
        
        # coarse-grained
        if w_best: 
            if len(n_list) == 1: # division by 0 issue if length 1 list - equivalent to local
                w_err = player_error.loc['local'][j]
            else:
                w_err = (mue * (N-n) + var * sumsquares)/((N-n)*N + n*var*sumsquares/mue)
        else:
            w = w_list[j]
            w_err = mue * ( w**2/n + (1-w**2)/N) + ((1-w)**2/(N**2)) * sumsquares* var
        player_error.loc['coarse'][j] = w_err
        
        # fine-grained
        if v_best: 
            # calculate optimal v weights
            V_list = [var + mue/ni for ni in n_list]
            sum_inv = sum([1/Vi for Vi in V_list]) - 1/V_list[j]
            vjj = (1 + var * sum_inv)/(1 + V_list[j] * sum_inv)
            weights = [(V_list[j]-var)/(Vk * (1 + V_list[j]*sum_inv)) for Vk in V_list]
            weights[j] = vjj
            v_vec = pd.DataFrame(weights)
        else:
            v_vec = pd.DataFrame(v_mat[j])
            
        player_error.loc['fine'][j] = (mue * (v_vec**2).T.dot(pd.DataFrame([1/nval for nval in n_list])) + 
                                       var * ((v_vec**2).sum() - v_vec.iloc[j]**2 + (1 - v_vec.iloc[j])**2))[0][0]
        
    return player_error

Next, in order to determine whether an arrangement is stable or not, we need to know the error each player would have gotten in any other coalition. The calc_error_groups() provides that calculation. 

In [4]:
def calc_error_groups(var = means_dist.var(), mue = variance_dist.mean(), same_size = False, 
                n = 10, M = 5, n_list = [1, 2]):
    '''
    Calculates the errors that players experience, for all possible arrangements of players into groups. 
    Assumes optimal versions of coarse and fine federation are used. 
    
    Args:
        var: variance of mean parameter. 
        mue: mean of errors. 
        same_size: boolean, indicates whether all players are the same size (if so, runs faster). 
        n: if players are the same size, indicates the size
        M: if players are the same size, indicates the number of players
        n_list: if players differ in their size, provides a list of those sizes. 
        
    Returns: 
        tables with errors for every combination of players (uniform, coarse, and fine-grained federation), 
        as well as local learning.
        
    
    '''
    
    if same_size:
        # all players are interchangable
        n_list = [n for i in range(M)]
        comb = list(itertools.combinations_with_replacement([0,1], r=M))
    else: 
        # for combinations, the identity of player matters (not interchangable)
        # for example: [1, 0, 1] means a group made of 1st and 3rd players only
        comb = list(itertools.product([0,1], repeat = len(n_list)))
    
    string_list = ["".join(map(str, val)) for val in comb] # string to name groups
    err_uniform = pd.DataFrame(data = np.nan, 
                              index = string_list,
                              columns = ['n_' + str(i) + '_err' for i in n_list] )
    err_best_coarse = err_uniform.copy()
    err_best_fine = err_uniform.copy()
        
    for index, group in enumerate(comb):
        loca = "".join(map(str, group)) # group index
        if sum(group) > 0: # ignore group with no members
            
            # drop players not in this group
            temp_n = [n_list[i] for i in range(len(n_list)) if list(group)[i] == 1]
            
            # calculate error table and rename columns
            error_table = calculate_means(var = var, mue = mue, n_list = temp_n, w_best = True, v_best = True)
            error_table.columns = [i for i in range(len(n_list)) if list(group)[i] == 1]  
            
            # copy errors into correct tables
            for player in range(len(n_list)):
                if list(group)[player] ==1: # if player is participating in group
                    err_uniform.iloc[index, player] = error_table.loc['uniform', player]
                    err_best_coarse.iloc[index, player] = error_table.loc['coarse', player]
                    err_best_fine.iloc[index, player] = error_table.loc['fine', player]
    
    local_error = calculate_means(var = var, mue = mue, n_list = n_list, w_best = True, 
                                  v_best = True).loc['local']
    
    err_uniform.dropna(how = 'all', inplace = True)
    err_best_coarse.dropna(how = 'all', inplace = True)
    err_best_fine.dropna(how = 'all', inplace = True)
    
    return err_uniform, err_best_coarse, err_best_fine, local_error

We can use the above function to calculate the error for all possible arrangements of players. Specifically, we can use this to recreate the tables in the "Motivating Example" section of the paper. 

Each column of the dataframe represents a single player and includes a note of how many samples they have. For example, "n_5_err" represents the error of a player with n=5 samples. Each row of the dataframe cooresponds to a description of the coalition in binary: 001 means the third player alone, 011 means the 2nd and 3rd players together, and so on. If a player is not participating ina coalition, their error for that coalition is NA. 

**Table 1**: uniform federation where all players have 5 samples each. Note that, for this table, all players minimize their error in the grand coalition. 

In [5]:
err_uniform, err_best_coarse, err_best_fine, local_error = calc_error_groups(var = 1, mue = 10, 
                                                                             n_list = [5, 5, 5])
err_uniform.replace([np.nan],['-'])

,n_5_err,n_5_err,n_5_err
001,-,-,2
010,-,2,-
011,-,1.5,1.5
100,2,-,-
101,1.5,-,1.5
110,1.5,1.5,-
111,1.33333,1.33333,1.33333


**Table 2**: uniform federation where two players have 5 samples and 1 player has 25 samples. Note that the arrangement where the two smaller players are federating together is the only stable one. 

In [6]:
err_uniform, err_best_coarse, err_best_fine, local_error = calc_error_groups(var = 1, mue = 10, 
                                                                             n_list = [5, 5, 25])
err_uniform.replace([np.nan],['-'])

,n_5_err,n_5_err,n_25_err
001,-,-,0.4
010,-,2,-
011,-,1.72222,0.388889
100,2,-,-
101,1.72222,-,0.388889
110,1.5,1.5,-
111,1.55102,1.55102,0.408163


**Table 3**: uniform federation where all 3 players have 25 samples. Note that all players minimize their error when they are doing local learning. 

In [7]:
err_uniform, err_best_coarse, err_best_fine, local_error = calc_error_groups(var = 1, mue = 10, 
                                                                             n_list = [25, 25, 25])
err_uniform.replace([np.nan],['-'])

,n_25_err,n_25_err,n_25_err
001,-,-,0.4
010,-,0.4,-
011,-,0.7,0.7
100,0.4,-,-
101,0.7,-,0.7
110,0.7,0.7,-
111,0.8,0.8,0.8


**Table 4**: coarse-grained federation where three players have 30 samples and the 4th has 300 samples. (Some values in the paper are rounded). Note that the grand coalition is individually stable, but not core stable: the three small players would prefer federating together to being in the grand coalition. 

In [8]:
err_uniform, err_best_coarse, err_best_fine, local_error = calc_error_groups(var = 1, mue = 10, 
                                                                             n_list = [30, 30, 30, 300])
err_best_coarse.replace([np.nan],['-'])

,n_30_err,n_30_err,n_30_err,n_300_err
0001,-,-,-,0.0333333
0010,-,-,0.333333,-
0011,-,-,0.286385,0.0328638
0100,-,0.333333,-,-
0101,-,0.286385,-,0.0328638
0110,-,0.291667,0.291667,-
0111,-,0.28279,0.28279,0.0326797
1000,0.333333,-,-,-
1001,0.286385,-,-,0.0328638
1010,0.291667,-,0.291667,-


**Table 5**: fine-grained federation where three players have 30 samples each and the 4th has 300 samples. (Again, some values in the paper are rounded). Note that the grand coalition is core stable. 

In [9]:
err_best_fine.replace([np.nan],['-'])

,n_30_err,n_30_err,n_30_err,n_300_err
0001,-,-,-,0.0333333
0010,-,-,0.333333,-
0011,-,-,0.286385,0.0328638
0100,-,0.333333,-,-
0101,-,0.286385,-,0.0328638
0110,-,0.291667,0.291667,-
0111,-,0.275327,0.275327,0.0326797
1000,0.333333,-,-,-
1001,0.286385,-,-,0.0328638
1010,0.291667,-,0.291667,-


Next, we include subroutines that take in a dataframe as produced by calc_error_groups(), as well as a current state. For example, calculating whether an arrangement is stable or not requires asking whether there exists an coalition where all players do strictly better: a list of current errors is sufficient to check this, along with the dataframe from calc_error_groups(). Conversely, for calculating whether an arrangement is individually stable, we need to check whether a single player would join any of the existing coalitions: for this, a vector of current errors and a list of the current coalitions (a coalition structure) is needed. 

In [10]:
def check_core(err, coalition):
    '''
    Given a coalition and a dataframe of all possible combinations of players, calculate any deviations under
    core or strict core. 
    
    Args: 
        err: a dataframe of errors (rows are coalitions, columns are players). 
        coalition: a reference list of errors: we are checking this for stability. 
        
    Returns: 
        dataframe with rows for each deviating coalition (empty if none possible). 
    '''
    eps = 0.0001 # to handle numerical precision
    # for core stability: all deviating players must strictly benefit
    core = err[np.all((err - coalition < -eps) | np.isnan(err), axis = 1)]
    # for strict core stability: all deviating players must weakly benefit, with at least one strictly benefiting
    strict_core = err[np.all((err <= coalition) | np.isnan(err), axis = 1) & 
                     np.any(err - coalition < -eps, axis = 1)]
    return core, strict_core

def check_IS(err, coalition, col_struct, stop_if_not_stable = False):
    '''
    Given a coalition and a dataframe of all possible combinations of players, calculate any deviations under
    individual stability. (An arrangement is stable under individual stability if no players wish to move to 
    any coalition that would weakly benefit from having them join). Much slower than core stability to check: 
    depends not only on the current error of players, but their current arrangement. 
    
    Args: 
        err: a dataframe of errors (rows are coalitions, columns are players). 
        coalition: a reference list of errors: we are checking this for stability. 
        col_struct: the current coalition structure 
        stop_if_not_stable: boolean. If true, stop and return values as soon as a deviation coalition is found. 
        
    Returns: 
        a dataframe with rows for each deviating coalition (empty if none possible). 
    '''
    eps = 0.0001  # to handle numerical precision
    moves = [] # list of all possible moves
    M = err.shape[1] # number of players
    for player in range(1, M+1):
        # check if player would rather be by themselves
        if np.nansum(err.loc[[(player,)]]) - coalition[player-1] < -eps:
            moves.append((player,))
        # for every coalition structure the player isn't in, see if they wish to move (and others let them)
        for col in col_struct:
            # early stopping
            if stop_if_not_stable & len(moves)>0:
                return err.loc[moves].drop_duplicates()
            if player not in col:
                # create new coalition 
                new_val = tuple(sorted(col + (player,)))
                new_coalition = err.loc[[new_val]]
                # check if condition holds
                if (np.all((new_coalition <= coalition) | np.isnan(new_coalition)) & 
                    (new_coalition.values[0][player-1] - coalition[player-1] < -eps)):
                    moves.append(new_val)
    return err.loc[moves].drop_duplicates()

Finally, calc_stability() takes in a set of inputs (a tuple of players with various number of samples, or else a list of tuples of players, to check multiple arrangements) and returns a dataframe indicating whether or not each arrangement is stable. 

In [11]:
def calc_stability(var = 1, mue = 10, fed_strat = 'uniform', M=3, n_tuples_list = None, return_w = False, 
                   stability = 'core', print_val = False, only_grand = False, IS_stop_if_not_stable = True, 
                   stop_once_stable = False):
    """
    For a given set or sets of players, calculate which, if any, are stable (core stable, nash stable, 
    individually stable). Note: this is a very slow function, especially for individually stable. 
    
    Args: 
        var: variance of mean parameter. 
        mue: mean of errors. 
        fed_strat: 'uniform', 'coarse', or 'fine': indicates which federation method is used. (optimal versions)
        M: indicates the number of players (overridden by n_tuples_list). 
        n_tuples_list: a list of tuples of combinations of n values (sample sizes) to check. If "None", uses 
                       default. 
        stability: 'core', 'core_strict', or 'IS': type of stability to check for. 
        print_val: boolean, if true, print values for each tuple of n values checked if not stable.  
        only_grand: boolean, if true, only check whether the grand coalition is stable. 
        IS_stop_if_not_stable: boolean, for IS subfunction: stop calculating deviations once an arrangement is 
                               proven unstable. 
        stop_once_stable: boolean, overall: stop once a single stable coalition structure is found. 
        
    Returns: 
        A dataframe indicating which arrangements are stable: rows are players (number of samples each has)
        and columns are coalition structures. 
        
    """
    
    if n_tuples_list == None:
        n_range = [1] + list(np.arange(0, 25, 5)[1:])
        # gives unique combinations of sizes
        n_tuples_list = list(itertools.combinations_with_replacement(n_range, r = M))
        # gives repeats: eg [1, 5, 10] and [5, 1, 10] both present
        # n_tuples_list = list(itertools.product(n_range, repeat = M))
    else: 
        # number of elements in n_tuples_list overrides M value provided.
        M = len(n_tuples_list[0])
    
    # create dictionary mapping to groups
    M_list = list(range(M + 1))[1:]
    w0,_, _, _ = calc_error_groups(n_list = M_list) # to get index 
    dict_val_rev = {val: tuple(ind+1 for ind, binary in enumerate(val) if binary == '1') for val in w0.index}
    
    if only_grand: 
        # only check grand coalition. 
        col_list = [(tuple(M_list),)]
    else:
        # get list of partitions
        col_list = [part for k in range(1, len(M_list) + 1) for part in set_partitions(M_list, k)]
    

    stab_df = pd.DataFrame(data = np.nan, 
                           index = n_tuples_list,
                           columns = col_list)

    for ind_n, n_tuple in enumerate(stab_df.index):
        
        # calculate error matrix, select correct one
        uniform_err, coarse_err, fine_err, _ = calc_error_groups(n_list = list(n_tuple), var = var, mue = mue)
        if fed_strat =='uniform':
            err = uniform_err
        elif fed_strat == 'coarse':
            err = coarse_err
        elif fed_strat == 'fine':
            err = fine_err
        err.index = [dict_val_rev[val] for val in err.index] # rename index
        
        # for each coalition we are checking for stability: 
        for ind_col, col in enumerate(col_list):
            # create a list of errors each player gets
            coalition = np.nansum(err.loc[list(col)], axis=0)
            if stability == 'core':
                devs, _ = check_core(err, coalition)
            elif stability == 'core_strict':
                _, devs = check_core(err, coalition)
            elif stability == 'IS':
                devs = check_IS(err, coalition, col, IS_stop_if_not_stable)
            stab_df.iloc[ind_n, ind_col] = (len(devs) ==0)
            if stop_once_stable & (len(devs)==0):
                return core_df, err
            if print_val:
                if len(devs) >0:
                    print(n_tuple)
                    print(col)
                    print(coalition)
    return stab_df

For example, we can calculate the core stable arrangement in Table 4 easily: the only stable arrangement is to have the three small players grouped together, while the fourth large player is by itself. This is represented by ((1,2, 3), (4,)). 

In [12]:
calc_stability(fed_strat = 'coarse', stability = 'core', n_tuples_list = [(30, 30, 30, 300)]).replace(
    [True, False],['Stable', 'Not'])

,"((1, 2, 3, 4),)","((1,), (2, 3, 4))","((2,), (1, 3, 4))","((3,), (1, 2, 4))","((4,), (1, 2, 3))","((1, 2), (3, 4))","((1, 3), (2, 4))","((1, 4), (2, 3))","((1,), (2,), (3, 4))","((1,), (3,), (2, 4))","((1,), (4,), (2, 3))","((2,), (3,), (1, 4))","((2,), (4,), (1, 3))","((3,), (4,), (1, 2))","((1,), (2,), (3,), (4,))"
"(30, 30, 30, 300)",Not,Not,Not,Not,Stable,Not,Not,Not,Not,Not,Not,Not,Not,Not,Not


We can also explore stablility for series of situations. For example, suppose we fix the number of players $M=3$ and say that they all have the same number of samples $n$, but vary $n$. We can get the results below, showing that the grand coalition ((1, 2, 3),) is core stable when $n\leq 10$ and local learning is core stable for $n>10$. 

In [13]:
n_range = [1] + list(np.arange(0, 25, 5)[1:])
n_tuples_list_same = list(tuple([n]*3) for n in n_range)

In [14]:
calc_stability(fed_strat = 'uniform', stability = 'core', n_tuples_list = n_tuples_list_same).replace(
    [True, False],['Stable', 'Not'])

,"((1, 2, 3),)","((1,), (2, 3))","((2,), (1, 3))","((3,), (1, 2))","((1,), (2,), (3,))"
"(1, 1, 1)",Stable,Not,Not,Not,Not
"(5, 5, 5)",Stable,Not,Not,Not,Not
"(10, 10, 10)",Stable,Stable,Stable,Stable,Stable
"(15, 15, 15)",Not,Not,Not,Not,Stable
"(20, 20, 20)",Not,Not,Not,Not,Stable


We can also run the same values for coarse-grained federation and fine-grained federation. Note that the grand coalition is always individually stable for coarse-grained federation and always core stable for fine-grained federation. 

In [15]:
calc_stability(fed_strat = 'coarse', stability = 'IS', n_tuples_list = n_tuples_list_same).replace(
    [True, False],['Stable', 'Not'])

,"((1, 2, 3),)","((1,), (2, 3))","((2,), (1, 3))","((3,), (1, 2))","((1,), (2,), (3,))"
"(1, 1, 1)",Stable,Not,Not,Not,Not
"(5, 5, 5)",Stable,Not,Not,Not,Not
"(10, 10, 10)",Stable,Not,Not,Not,Not
"(15, 15, 15)",Stable,Not,Not,Not,Not
"(20, 20, 20)",Stable,Not,Not,Not,Not


In [16]:
calc_stability(fed_strat = 'fine', stability = 'core', n_tuples_list = n_tuples_list_same).replace(
    [True, False],['Stable', 'Not'])

,"((1, 2, 3),)","((1,), (2, 3))","((2,), (1, 3))","((3,), (1, 2))","((1,), (2,), (3,))"
"(1, 1, 1)",Stable,Not,Not,Not,Not
"(5, 5, 5)",Stable,Not,Not,Not,Not
"(10, 10, 10)",Stable,Not,Not,Not,Not
"(15, 15, 15)",Stable,Not,Not,Not,Not
"(20, 20, 20)",Stable,Not,Not,Not,Not
